In [1]:
%%html
<style>
table {float:left}
</style>

*note: It's come to my attention that Wordpress is truncating some of my posts so that the right hand side is blocked by the side bar. My apologies for the this. I'll get it fixed ( or more likely move to GH pages ) as quickly as possible. Thanks for your patience*

*@netmanchris*


# Deploying Code to Devices Through your NMS

If you're luck enough to have an NMS as powerful as [HPE IMC](http://www.hpe.com/networking/imc) then you already have a very capable system which has a ton of APIs that you probably didn't even know about. IMC isn't the only NMS which has APIs these days, but it's the one we're going to be looking at here. 

We've spent the last few posts ( [here](https://kontrolissues.wordpress.com/2016/01/14/intro-to-jinja2/), [here](https://kontrolissues.wordpress.com/2016/01/19/more-jinja-working-with-yam-as-an-input/), and [here](https://kontrolissues.wordpress.com/2016/01/29/git-and-jinja-like-peanut-butter-and-pickles/]) running through creating some network configurations through the [Jinja2 templating language](http://jinja.pocoo.org/docs/dev/). 

There are at least a couple of immediate benefits to this approach:
- Consistency in the configuration between devices
- Accuracy in the commands going into your devices


But the one large draw back is that you've **still** got to cut and paste that configuration into your device somehow, which is not the ideal scenario. We're trying to get away from touching our devices. 

In this post, we're going to look at taking the rendered configuration and pushing it directly to the desired device through HPE IMC's RESTful API, refered to as the eAPI in documentation. 

*Although there used to be a charge for this, HPE recently made some changes and the RESTful API is now included in both the Standard and Enterprise editions of the NMS.*



In [2]:
import requests
import yaml
import githubuser
from pyhpeimc.auth import *
from pyhpeimc.plat.device import *
from pyhpeimc.plat.icc import *
from pyhpeimc.plat.vlanm import *
from jinja2 import Environment, FileSystemLoader, Template

# Loading the templates and values from Git

We've gone through this before, so I'm not going to spend much time here going over this. In a nutshell, we're loading the comware_template and the variables we'd like to use to fill in the template. Again, make sure you're using the **raw** URL from Github and not the normal URL or you will end up with the whole HTML structure and not just the content you're looking for. 



In [3]:
comware_template = requests.get('https://raw.githubusercontent.com/netmanchris/Jinja2-Network-Configurations-Scripts/master/comware_vlan.j2').text
gitauth = githubuser.gitcreds() #you didn't think I was going to give you my password did you?
simple = yaml.load(requests.get('https://raw.githubusercontent.com/netmanchris/Jinja2-Network-Configurations-Scripts/master/vlans.yaml', auth=gitauth).text)
cw_template = Template(comware_template)

# Rendering the template

Here we're going to take a quick look at the rendered combination of the comware_tempalte and the variables to make sure this is what we want to send during the final push to the device. Automation is great, but it's going to be a long time before it can replace a human being with knowledge of the environment. Trust... but verify. 

In [4]:
my_template = cw_template.render(simple=simple)
print (my_template)

#vlan config
vlan 1
    name default
    description default
vlan 2
    name TenantABC
    description TenantABC
vlan 3
    name management
    description management
vlan 10
    name mgmt
    description mgmt



#  Options, Options, Options...

We now have a decision to make. There are a couple of different APIs available to us to push VLANs to the device.

For this example, we're going to use the executecmd API that allows us to send a series of commands to the device through the HPE IMC REST API. 



![vlan api](./images/executecmd.png)


As you can see from the REST documentation, you need to send a JSON object which is a list of the commands that you would type in from the command prompt of the switch.

So there are a couple of things we need to prepare the rendered jinja template into a format that can be sent to the API. 

1. We need to add the command <span style="color:blue"> "system-view" </span> to the beginning of the command list. 
> system-view on HPE Comware devices is equivalent to the enable + conf t commands using the IOS syntax you're probably used to

2. We need to split the giant string that rendering the jinja template gave us into a python list with one command per list item. Thankfully, we can use the python <span style="color:blue">split</span> method to help us through this. We can use the carriage return symbol to identify the end of each line. python identifies the carriage return by the <span style="color:blue">\n</span> characters which is what we're going to use as the input to the split method.

3. Once we've got those two things done, we simply add the two together and voila!


In [5]:
cmd_list = ['system-view']
cmd_list = cmd_list + my_template.split('\n')


##Trust but verify

Are you seeing a trend here? If we're ever going to learn to trust automation, we need to get comfortable that our expectations are met at each step of the journey, so we're going to take a look at the new **cmd_list** variable and make sure that 

- it's a list
- the first elemend of the list is **system-view**
- the rest of the list is one command per element
- all the commands are in the right order

In [6]:
cmd_list[0:10]

['system-view',
 '#vlan config',
 'vlan 1',
 '    name default',
 '    description default',
 'vlan 2',
 '    name TenantABC',
 '    description TenantABC',
 'vlan 3',
 '    name management']

# Sending the commands

So far, other than splitting on the \n, this isn't much different than what we've covered in the other blog posts. Now is where we connect the list of commands we've created to the device they are destined for. 

The first thing we're going to do is to create an authentication object that we can use to feed into the requests commands upon sending to the REST API. 

In [7]:
auth = IMCAuth("http://", "10.101.0.203", "8080", "admin", "admin")
 

# Getting the Device ID

The input for the run_dev_cmd is the device ID. This is an internal number that IMC uses to idenitfy that specific device. Thankfully, we've also got an RESTful function to get that based on the IP address of the device. To make things a little bit easier on us, we will grab the results of the get_dev_details API and assign the device ID directly to a variable called **devid**. Once we've got the device ID back, this gives us what we need to move on to the next steps. 


In [8]:
devid = get_dev_details('10.20.10.10', auth.creds, auth.url)['id']
devid

'221'

# Sending the Commands to the target Device

We will now use the **run_dev_cmd** function from the **pyhpeimc** library to send the commands directly to the device. You can see that we are using the **devid** variable assigned above as the input for the target device. We're also using the **cmd_list** variable that containts the list of all the commands that we wish to send to the device.

We're going to look for the contents of the **success** response. Which, if we're lucky, should be **true**. 

In [9]:
run_dev_cmd(devid, cmd_list, auth.creds, auth.url)['success']

'true'

# Double Checking the VLANs

Now that we've sent the VLANs to the device, the last thing we should be doing is to double check that nothing went wrong in the sending. We'll use the exact same **run_dev_cmd** function, but this time, we'll be sending the **display vlan** command and looking at the **content** of the return instead of the **success**. 

In [10]:
cmd_list = ['system-view', 'display vlan']
print (run_dev_cmd(devid, cmd_list, auth.creds, auth.url)['content'])



 1(default), 2-3, 5, 10 


# Getting better, right?


So we've come a long way in a short time. We've 
- started with an introduction to [Jinja2 templating language](https://kontrolissues.wordpress.com/2016/01/14/intro-to-jinja2/)
- learned how to [access templates and input variables from our hard drive](https://kontrolissues.wordpress.com/2016/01/19/more-jinja-working-with-yam-as-an-input/)
- [Moved our access tempaltes and input variables to GitHub so that we can work collabaratively with our team](https://kontrolissues.wordpress.com/2016/01/29/git-and-jinja-like-peanut-butter-and-pickles/)

And in this post, we learned how to leverage the first three to deploy configurations directly from code to our devices. 

## The good part

For those who have done some scripting to device before, you'll have noticed that using an API provided by an NMS such as [HPEIMC](http://www.hpe.com/networking/imc) makes life much easier. We didn't have to worry about username and passwords for the individual devces, nor having to worry about deciding what protocol we need to use to connect to the device. The great part about using the NMS as a proxy is that all the credential and protocl negotiations are all handled by the NMS itself, allowing us to get on to the trouble of worrying about what we want to send to our devices and not concerning with how they actually get there. 


This is a big step forward, but there are still a couple of small problems that we need to address

## Configuration Drift

If you look closely, we've actually got an extra VLAN in there. **VLAN 5** has been configured on the device, but it's not in our list of desired_vlans where we have declared exactly which VLANs should be on the target device. This is what is sometimes known as configuration drift. Some people may say 

> Hey, It's just an extra VLAN right? That won't hurt us!

Sorry to respectfully disagree, but this attitude is exactly what causes us issues. This is the death of a thousand cuts. It's **JUST** one VLAN, **JUST** one switch running a differnet version of code, **JUST** one router that has some unused sub-interfaces on it.

**IT'S JUST ONE MORE THING THAT WILL BITE YOU WHEN YOU'RE TROUBLESHOOTING AN ISSUE.**

These **JUST** things are what we sometimes call technical debt. If you can figure out out. 

## Vendor Syntax

The other problem with this example is that we are bound to a specific vendor's syntax. If you attmept to run the **system-view** command on a Juniper/Cisco/Brocade/Extreme/ARISTA device, it's going to error out. Right?  This coule easily be addressed by some conditional logic which figures out which kind of a box it is first and then leverages a specific Jinja template for that vendor, but you can see how this becomes a slippery slope rather quickly. 

In the next post, we're going to look at a way to address both of these issues. 

Stay Tuned!

@netmanchris


P.S. As always, comments and questions are more than welcome. 